# Model Context Protocol (MCP) Server Example 
This Jupyter notebook runs on Colab and demonstrates a MCP server running Mellea. 

## Install Ollama

Before we get started with Mellea, we download and install ollama.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &

## Install Mellea
We run `uv pip install .` to install Mellea.

In [ ]:
import os
!git clone https://github.com/generative-computing/mellea.git --quiet
os.chdir("mellea")
!uv pip install . -qq

## Import Mellea and MCP Server

In [ ]:
from mcp.server.fastmcp import FastMCP
from mellea import MelleaSession
from mellea.backends import model_ids
from mellea.backends.ollama import OllamaModelBackend
from mellea.stdlib.base import ModelOutputThunk
from mellea.stdlib.requirement import Requirement, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

## Create an MCP Server
Run MCP debug UI with: `uv run mcp dev docs/examples/mcp/mcp_example.py`. We can wrap a simple mcp server around a program and use the server as-is. 

In [ ]:
mcp = FastMCP("Demo")

@mcp.tool()
def write_a_poem(word_limit: int) -> str:
    """Write a poem with a word limit."""
    m = MelleaSession(OllamaModelBackend(model_ids.QWEN3_8B))
    wl_req = Requirement(
        f"Use only {word_limit} words.",
        validation_fn=simple_validate(lambda x: len(x.split(" ")) < word_limit),
    )
    res = m.instruct(
        "Write a poem",
        requirements=[wl_req],
        strategy=RejectionSamplingStrategy(loop_budget=4),
    )
    assert isinstance(res, ModelOutputThunk)
    return str(res.value)

@mcp.resource("greeting://{name}")
def get_greeting(name: str) -> str:
    """Get a personalized greeting"""
    return f"Hello, {name}!"    